# Лобанова Валерия 310

### Инкубационный период в Китае и в остальной Азии

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
from datetime import datetime

In [3]:
data = pd.read_csv ("covid_short_data-5.csv")

#Asia_countries without 'China'
Asia_countries  = ['Cambodia', 'Nepal', 'South Korea', 'Singapore', 'Thailand', 'Hong Kong', 'Lebanon', 'Phillipines', 'Afghanistan', 'Kuwait', 'Iran', 'Taiwan', 'India',  'Israel', 'UAE', 'Malaysia', 'Bahrain', 'Russia', 'Japan', 'Vietnam', 'Sri Lanka']

In [4]:
def to_date (date):
    if len (date.split('/')[-1]) == 2:
        return datetime.strptime (date, '%m/%d/%y').date ()
    return datetime.strptime (date, '%m/%d/%Y').date ()

def numOfDays (date1, date2): 
    return (to_date (date2) - to_date (date1)).days

def IncubationPeriod (data, m, n, list_countries = ['China']):
    if type (data['exposure_end'][n]) == str \
        and type (data['symptom_onset'][n]) == str \
        and data['country'][n] in list_countries:
            nD = numOfDays (data['exposure_end'][n], data['symptom_onset'][n])
            if nD >= 0:
                return nD + m
    elif type (data['exposure_start'][n]) == str \
        and type (data['symptom_onset'][n]) == str \
        and data['country'][n] in list_countries:
            nD = numOfDays (data['exposure_start'][n], data['symptom_onset'][n])
            return nD - m
    return 

In [5]:
delta_days = np.array ([numOfDays (data['exposure_start'][n], data['exposure_end'][n]) 
                        for n in range (len (data['reporting date'])) 
                            if type (data['exposure_start'][n]) == str 
                            and type (data['exposure_end'][n]) == str ])

median = np.median (delta_days)

In [6]:
China_incubation = np.array ([IncubationPeriod (data, median, n) 
                        for n in range (len (data['reporting date'])) 
                            if IncubationPeriod(data, median, n) is not None])

Asia_incubation = np.array ([IncubationPeriod (data, median, n, Asia_countries) 
                        for n in range (len (data['reporting date'])) 
                            if IncubationPeriod(data, median, n, Asia_countries) is not None])  

Shapiro of China_incubation:

In [7]:
stats.shapiro (China_incubation)

ShapiroResult(statistic=0.8756677508354187, pvalue=7.205451765912585e-06)

Shapiro of Asia_incubation:

In [8]:
stats.shapiro (Asia_incubation)

ShapiroResult(statistic=0.7994221448898315, pvalue=1.5635661654300748e-09)

$p_{value} < 0.05\;$ в обоих выборках $\Rightarrow$ нормальность отвергается


In [9]:
print ("Size of China_incubation = %d" % len (China_incubation))
print ("Size of Asia_incubation  = %d" % len (Asia_incubation))

Size of China_incubation = 67
Size of Asia_incubation  = 87


Выборки достаточно большие для применения асимптотического двухвыборочного критерия Вилкоксона

In [10]:
stats.ranksums(China_incubation, Asia_incubation)

RanksumsResult(statistic=-2.973842097692456, pvalue=0.002940963732477172)

Вывод: распределение инкубационных периодов Китая и остальных стран Азии отличаются